In [5]:
# running geneformer and deepSem

from geneformer import EmbExtractor
from geneformer import TranscriptomeTokenizer

import anndata as ad
import torch
import scanpy as sc

from bengrn import BenGRN, get_perturb_gt
from grnndata import GRNAnnData
from grnndata import utils as grnutils

%load_ext autoreload
%autoreload 2

FOLDER = "../../../Geneformer/"
NUM_GENES=5000


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import requests

# Create a directory to store the downloaded files
os.makedirs("../../data/human_dcm_hcm_nf", exist_ok=True)

# URLs for the files to download
base_url = "https://huggingface.co/datasets/ctheodoris/Genecorpus-30M/resolve/main/example_input_files/cell_classification/disease_classification/human_dcm_hcm_nf.dataset/"
files_to_download = ["dataset.arrow", "dataset_info.json", "state.json"]

# Download the files
for file in files_to_download:
    url = base_url + file
    output_path = os.path.join("../../data/human_dcm_hcm_nf", file)
    
    if not os.path.exists(output_path):
        print(f"Downloading {file}...")
        response = requests.get(url)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"\nDownloaded {file} to {output_path}")
        else:
            print(f"Failed to download {file}. Status code: {response.status_code}")
    else:
        print(f"{file} already exists in {output_path}")

print("Download complete!")



dataset.arrow already exists in ../../data/human_dcm_hcm_nf/dataset.arrow
dataset_info.json already exists in ../../data/human_dcm_hcm_nf/dataset_info.json
state.json already exists in ../../data/human_dcm_hcm_nf/state.json
Download complete!


In [2]:
genes = torch.load('../../data/temp/vbd8bavn/epoch=17-step=90000.ckpt')['hyper_parameters']['genes']

/tmp/ipykernel_2132580/1970222934.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  genes = torch.load('../../data/temp/vbd8bavn/epoch=17-step=90000.ckpt')['hyper_paramete

In [3]:
from geneformer import TOKEN_DICTIONARY_FILE
import pickle

with open(TOKEN_DICTIONARY_FILE, "rb") as f:
    gene_mapping_dict = pickle.load(f)

In [4]:
adata = get_perturb_gt()

adata.var["isTF"] = False
adata.var.loc[adata.var.gene_name.isin(grnutils.TF), "isTF"] = True
adata.var["isTF"].sum()

/home/ml4ig1/Documents code/benGRN/bengrn/base.py:624: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_sc.obs["organism_ontology_term_id"] = "NCBITaxon:9606"


NameError: name 'grnutils' is not defined

In [7]:
adata.var['ensembl_id'] = adata.var.index
adata.obs['n_counts'] = adata.X.sum(1)

In [9]:
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=4000)
subadata = adata[:, adata.var.index.isin(genes) & adata.var.highly_variable]

In [12]:
subadata[:, subadata.var.index.isin([u for u in gene_mapping_dict.keys() if u in subadata.var.index])]

In [13]:
subadata.write_h5ad("test.h5ad")

/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/anndata/_core/anndata.py:1145: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


In [ ]:
genelist = [gene_mapping_dict[u] for u in subadata.var.index]

In [14]:
tk = TranscriptomeTokenizer(nproc=16)
tk.tokenize_data("./", 
                 "../../data/",
                 "tokenized_data", 
                 file_format="h5ad")

Tokenizing test.h5ad
test.h5ad has no column attribute 'filter_pass'; tokenizing all cells.


/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:473: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:476: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


Creating dataset.


In [ ]:
avg_attentions = torch.mean(torch.stack(attentions), dim=(0, 2))

In [ ]:
filtered_input_data.features['input_ids']

In [17]:
embex = EmbExtractor(model_type="CellClassifier",
                     num_classes=3,
                     filter_data={"cell_type":["Cardiomyocyte1","Cardiomyocyte2","Cardiomyocyte3"]},
                     max_ncells=1000,
                     emb_layer=0,
                     emb_mode="cell",
                     emb_label=["disease","cell_type"],
                     labels_to_plot=["disease"],
                     forward_batch_size=10,
                     nproc=16)


In [ ]:
embex